In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

In [2]:
torch.manual_seed(1337)
batch_size = 32
context_length = 8
learning_rate = 1e-3
max_iters = 3000

In [3]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
print(f">> Number of characters : {len(text)}")

>> Number of characters : 1115394


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# These are all the unique characters which occur in our text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# Tokenizing the text into integers by creating mapping
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hii There!"))
print(decode(encode("Hii There!")))

[20, 47, 47, 1, 32, 46, 43, 56, 43, 2]
Hii There!


In [8]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Splitting into train-val dataset
train_len = int(0.9 * len(data))
train_data = data[:train_len]
val_data = data[train_len:]

In [10]:
# Example of how our model will predict given x as input
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1]
    pred = y[t]
    print(f">> For the input {context}, the output is {pred}")

>> For the input tensor([18]), the output is 47
>> For the input tensor([18, 47]), the output is 56
>> For the input tensor([18, 47, 56]), the output is 57
>> For the input tensor([18, 47, 56, 57]), the output is 58
>> For the input tensor([18, 47, 56, 57, 58]), the output is 1
>> For the input tensor([18, 47, 56, 57, 58,  1]), the output is 15
>> For the input tensor([18, 47, 56, 57, 58,  1, 15]), the output is 47
>> For the input tensor([18, 47, 56, 57, 58,  1, 15, 47]), the output is 58


In [11]:
# Generating batches of inputs
def get_batch(split):
    data = train_data if split == "train" else val_data
    idxs = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in idxs])
    y = torch.stack([data[i+1:i+context_length+1] for i in idxs])
    
    return x, y

xb, yb = get_batch("train")
print("inputs : ")
print("Shape : ", xb.shape)
print("X : ", xb)
print("outputs : ")
print("Shape : ", yb.shape)
print("y : ", yb)

for b in range(batch_size):
    for t in range(context_length):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context}, the target is {target}")

inputs : 
Shape :  torch.Size([32, 8])
X :  tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1,

The BigramLanguageModel is a very simple model that learns to predict the next word just by looking at the previous one. 

So if you give it a word like "cat", it might learn that the next word could be "is" or "sat". 

But it’s a bigram model — meaning it only looks at one token at a time to guess the next one. It’s like playing a guessing game with one eye closed!

```nn.Embedding(vocab_size, vocab_size)``` creates a table of random numbers.

It does not know anything yet.

During training:

- The model looks up a token's row (embedding).

- It treats that row as the guess (logits) for the next token.

- It compares with the true next token (target).

- It adjusts the table values so future guesses improve

In [12]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads the logits for the next token from a look up table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # Both idx and targets are tensors of integers of shape (Batch, Time)
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel), channels = vocab size
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # shape (N, C) → N = number of samples (each with C scores)
            targets = targets.view(B*T) # shape (N,) → N class indices (0 to C−1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is the (B, T) array of indices for the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # Convert into (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Concatenate the prediction into the original input idx
            idx = torch.cat([idx, idx_next], dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(logits.shape)
print(loss)

idx = torch.zeros([1,1], dtype=torch.long) # Initial input is just 0, 0 -> \n
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.6922, grad_fn=<NllLossBackward0>)

JLg,3D&OM .3YCjfolRwqXaDyttW!GmaUT-IIvuZV?sYfjzUvTQ3RwL ?etyLeg.COHW
Ri$ELkJMXpBEX;-G&Orl!bcH ;cq.z,


In [13]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [14]:
# training loop
for iter in range(max_iters):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
loss.item()

2.9177985191345215

In [15]:
idx = torch.zeros([1,1], dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))


The PWou w keem'arePOn?kepo:LgesKWDanEThoThest he,F? hancoCour a bs fryss.-Fk, hmbo thwiXcTharh XnAbrezQG hid'veBocENe fjomurXFu bs cvn tlashwha;-z-k ir noniqYCr?VTheNerinc?BHicBONEMVDro haisrY$P-UcN;Pbym, il?IHag hilksF!bey fr l:QN&.
MESoCO:X;, harior haUCYOdqc?HESD wilag&thecoucQ's, s;
A aircald anHay'dG t:FLTOLAnow wz;'zA:yBmz; al,Nxfowpan:horouKmysbunazonbnlyme ired
Shire m wireamPur:tBr:
SUD:
Tha
Seum VTDKJ&ily gyoG&kemy: tUCEk'zSe
OK:m,Dqq!orkOKJkELve wmure Edy.xpELTano hwkllPSlonncg b? arenjz&-
LTZPSe c?nicusoo belq bjRCan oqGomWarifrcni3Ou wblolecl, oiremEirencalnhoAcyoujTALEXns,r maz:Zb?'to n! heB; t,J$YpR t e! .
SKJByw kG:
DWSThalqu-NG&BUBe aivad es&MIm b;
F$g
usOJGio
WhtuldesembeYBean,CcoNGHowe,d,CYce ss:
IIXtJ:cle e aomedod tJfun w?vevLA:-agiab':
jur, reiud h'm,FA:

F-car t, umaresth, h, rjllfCBlocoCe,,UThXld bu anonser,'Thng.xLgKp-qavu nEMELgKYin dengtoCado:
QpRCcu hangvsth,pwQg hthveZzik iskTu iljZWoduf's?J$bad t wrld han hxUd-eBq!-y heity, t gsIFKx, nistikOcastmicooCr b

# The Mathematical trick of self attention

In [16]:
# consider the following toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn((B,T,C))
x.shape

torch.Size([4, 8, 2])

In [48]:
# Each time-step can communicate with only the previous time-steps, so we will take mean of channels of prev time-steps including current
# We want x[b,t] = mean_{i <= t} x[b,i]
# Version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [51]:
# We can make it more efficient by weighted matrix multiplication
# Version 2 -> Matrix Multiplication
weights = torch.tril(torch.ones(T, T)) # Lower triangular matrix of weights bcz at each time step we can only look at time steps before it
weights = weights / weights.sum(1, keepdim=True) # Averaging the weights, so that sum of each row is 1
xbow2 = weights @ x # (B, T, T) @ (B, T, C) --> (B, T, C)

# We can see that xbow and xbow2 are exactly same
torch.allclose(xbow, xbow2, atol=1e-6)

True

In [52]:
# Version 3 -> Using softmax
tril = torch.tril(torch.ones(T, T))
weights = torch.zeros(T, T)
weights = weights.masked_fill(tril == 0, float('-inf')) # Masking all the future tokens with -inf
weights = F.softmax(weights, -1) # softmax(-inf) = 0, and all the ones will be averaged
xbow3 = weights @ x

# We can see that xbow3 is similar to xbow
torch.allclose(xbow, xbow3, atol=1e-6)

True

# Applying self-attention

In [59]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn((B, T, C))

# Let's see a single head of self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)

weights = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)
tril = torch.tril(torch.ones((T, T)))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, -1)
out = weights @ v

print(out.shape)

torch.Size([4, 8, 16])


In [60]:
# Now weights is not the same for each token
weights

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089